In [68]:
# for first part

import pandas as pd
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import numpy as np
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel
import re
import unidecode
from gensim.models.phrases import Phrases, Phraser

#download puntuation and stopwords
nltk.download("punkt")
nltk.download("stopwords")

#additional for second part

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\carme\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
#pip install Unidecode

### Import the cleaned data

In [69]:
data = pd.read_csv('C:/Users/carme/Dropbox/MIO/DOCS/Hertie/Thesis/Hertie-Thesis/Data/clean_data.csv')
# 'C:/Users/carme/Dropbox/MIO/DOCS/Hertie/Thesis/Elecciones_Municipales_2024/clean_data.csv'
#filter column with na values
plans = data.copy().dropna()
plans = plans.reset_index(drop=True)
plans.shape

(310, 10)

Delete the one duplicate value

In [70]:
# Encontrar duplicados en 'clean_text'
duplicated_texts = plans['clean_text'][plans['clean_text'].duplicated(keep=False)]

# Mostrar los documentos duplicados
# print(duplicated_texts)

# plans.loc[303,'clean_text']
plans.at[303,'clean_text'] = float("NaN")
# plans.loc[303,'clean_text']
plans = plans.dropna()
plans.shape

(309, 10)

Remove accents

In [71]:
#remove accents from spanish words
def remove_accents(text):
    return unidecode.unidecode(text)

plans['clean_text'] = plans['clean_text'].apply(remove_accents)

print(plans['clean_text'].head())

0    El ambiente nos importa\r\nLa preservacion del...
1        A Quienes Habitan Alajuela\r\n    Alajuela...
2                            PROGRAMA DE GOBIERNO\r...
3         PLAN DE GOBIERNO 2024 - 2028 | KATHIA ARR...
4    DESARROLLO\r\nECONOMICO..........................
Name: clean_text, dtype: object


Delete loose letters

In [72]:
# Definir la función para limpiar el texto
def limpiar_texto(texto):
    # Eliminar palabras de una sola letra que no sean 'a' o 'I' (ajustar según tu criterio)
    texto_limpio = re.sub(r'\b(?<!\b[aAiI]\b)[a-zA-Z]\b', '', texto)
    return texto_limpio

# Aplicar la función a cada elemento de la columna de texto
plans['clean_text'] = plans['clean_text'].apply(limpiar_texto)

To lower

In [73]:
# plans['clean_text'] = plans['clean_text'].lower()
plans['clean_text']=[text.lower() for text in plans['clean_text']]

Delete province, canton and name of candidate

In [74]:
# Función para eliminar palabras de las columnas especificadas de 'clean_text'
def remove_words(row, column_names):
    text = row['clean_text']
    for column in column_names:
        # Obtener palabras del campo actual, dividiendo por espacio o guión bajo
        # La expresión regular [ _] significa "espacio o guión bajo"
        # Usar str.lower() para convertir el texto a minúsculas antes de dividirlo
        words = re.split(r'[ _]', str(row[column]).lower())
        # Eliminar cada palabra, en sus diferentes formas de capitalización, del texto
        for word in words:
            text = re.sub(r'\b' + re.escape(word) + r'\b', '', text, flags=re.IGNORECASE)
    return text

    # Columnas de las cuales se eliminarán las palabras en 'clean_text'
columns_to_remove = ['Provincia', 'Municipalidad', 'Partido', 'Candidato']

# plans['clean_text'] = plans.apply(remove_words, column_names=columns_to_remove, axis=1)
plans.loc[:, 'clean_text'] = plans.apply(lambda row: remove_words(row, columns_to_remove), axis=1)

In [75]:
stop_words = set(stopwords.words('spanish'))
# Take accents from stopwords
stop_words = {unidecode.unidecode(word) for word in stop_words}

# Lista de palabras personalizadas para agregar
custom_stop_words = [
    'ana imelsa guzman espinoza','junnier a salazar tobal','claudia quintanilla tartaglione',
    'accion',
    'alajuelense', 'alcalde', 'alcalde', 'alcaldesa', 'alcaldia', 'alcaldia', 'ambito', 'anexo', 'anterior', 'apoyaremos', 'aqui',
    'brenes', 'busca', 'busca', 
    'cabo', 'cabo', 'canaza', 'candidata','canton','cantonal', 'carara', 'caso', 'cion', 'cl', 'continuar', 'continuar', 'coordinaremos', 'cosas', 'crearemos', 
    'cuales', 'cuanto', 'cuanto', 'cuatro', 'comunidad','comunidades',
    'debemos', 'debemos', 
    'elena', 'enrique', 'etc', 'etc', 
    'fomentaremos', 'formato', 
    'gestionaremos', 'golfito', 'guayabo', 
    'hagamoslo', 'haremos', 
    'ii', 'implementaremos', 'incluyendo', 'indicador',
    'juntos', 
    'kathia arroyo', 
    'linea', 'lineas', 'llevar', 
    'maria', 'meta', 
    'ndeg', 'numero',
    'objetivo', 
    'pagina', 'partir', 'periodo', 'permitira', 'pilar', 'plan de gobierno', 'poas', 'poasena', 'porcentaje', 'posible', 'posteriormente', 
    'primer', 'primera', 'promoveremos', 'proponemos', 'provincia', 'programa de gobierno','porque liberia debe avanzar',
    'responde', 'rodriguez', 
    'sa', 'samara', 'san', 'seccion', 'sino',
    'tabarcia', 'tal', 'tales', 'trabajaremos', 'tres', 'tribunal interno de elecciones',
    'vamos', 'vamos', 'ver',
    "ejecutarlo", "nombre", "finalidad", "complementado", "instrumento", "debera", "fondos", "asegurar",
    "verdes", "clave", "practicas", "potenciar", "asegurar", "habilidades", "ejes", "especial", "pueden", "nuevo", "asegurar",
    "metas", "tipo", "permanente", "falta" ,"requiere", "atenienses", "rapidos", "ateniense", "empiecen", "impulso", "conlleva", "validar",
    "colectivamente", "presta", "comprende", "orden", "anchas", "intereses" , "poner", "mencionamos" , "potenciar", "vote papeletas", "limonense",
    "creara", "rio_blanco", "limonenses", "resultados", "resumen", "iniciativa", "asegurando", "clave", "asegurar", "fecha", "descritos",
    "indicadores", "estrategico", "avanza", "continuaremos", "asimismo", "santa", "dia", "gracias", "realizadas", "aprovechamiento", "dad", "nes",
    "impulsaremos", "buscaremos", "des", "municipes", "tos", "desarrollaremos", "pavones", "elecciones", "esparcimiento", "realizacion", "total",
    "nuevo", "falta", "debido", "impulsaremos", "rutinario", "estrategicas", "menciona", "pag", "escazucenos", "rafael", "tes", "aportado",
    "sirviendo", "pusc", "propone", "merecemos", "construyendo", "tienden", "siempre", "ameriten", "requeriran", "treinta", "beneficiado", "cuento",
    "bicentenario"]


# Agregar las palabras personalizadas al conjunto de stopwords
stop_words.update(custom_stop_words)

#preprocess and stemming
# def preprocess(text):
#     return [word for word in word_tokenize(text.lower()) if word.
#             isalpha() and word not in stop_words]

# Spanish stemmer from Snowball
stemmer = SnowballStemmer("spanish")

#with stemming and preprocessing
def preprocess(text):
    # Normalize and remove non-alpha characters
    text = unidecode.unidecode(text.lower())
    text = re.sub(r'[^a-z ]+', '', text)
    # Tokenize, remove stopwords, and stem
    tokens = word_tokenize(text)
    # return ' '.join(stemmer.stem(word) for word in tokens if word not in stop_words) #with stemmer
    return ' '.join(word for word in tokens if word not in stop_words) #with no stemmer

plans['processed_text'] = plans['clean_text'].apply(preprocess)

Include bigrams

In [17]:
# # Construir modelos de bigramas y trigramas
# bigram = Phrases(processed_docs, min_count=10, threshold=10)  # Ajusta los parámetros según sea necesario
# bigram_mod = Phraser(bigram)

# # Aplicar el modelo para construir bigramas
# bigram_texts = [bigram_mod[doc] for doc in processed_docs]

Topic modeling:

In [10]:
# # Crear el diccionario y el corpus para LDA
# dictionary = Dictionary(bigram_texts)
# dictionary.filter_extremes(no_below=2, no_above=0.5)
# corpus = [dictionary.doc2bow(doc) for doc in bigram_texts]

Dictionary and corpus for LDA

In [174]:
# dictionary = Dictionary(processed_docs)
# dictionary.filter_extremes(no_below=2, no_above=0.5)
# corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

LDA: Training and analysis

In [44]:
# num_topics = 150  # Ajusta este número según tus necesidades
# lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=15)

# for i, topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=10):
#     print(f'Tema {i}: {topic}\n')

This is not great. Possible reasons:
+ lengths of documents are very different
+ the docs are not very different between each other

# Using TF-IDF. Otherwise we can try doing a Guided LDA.

In [47]:
# Initialize a TF-IDF Vectorizer
# The ngram_range parameter is set to (1, 2) to include both unigrams and bigrams
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, ngram_range=(1, 2))

# Transform the texts into a document-term matrix
dtm = tfidf_vectorizer.fit_transform(plans['processed_text'])

# Initialize and fit LDA
lda = LatentDirichletAllocation(n_components=30, random_state=0)
lda.fit(dtm)

# Function to print the topics and their top words
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic ", idx)
        print([(vectorizer.get_feature_names_out()[i], topic[i])
               for i in topic.argsort()[:-top_n - 1:-1]])

print_topics(lda, tfidf_vectorizer)

Topic  0
[('pensando grande', 0.3898240245831482), ('local areas', 0.2746915758651164), ('tacares', 0.21332041482546377), ('per capita', 0.20612819141733754), ('tasa mortalidad', 0.1993133864272263), ('capita', 0.19591388576373264), ('sarchi', 0.1891577129161759), ('municipaleje', 0.18012359259700947), ('ademas hacer', 0.17133341427346585), ('trabajo comites', 0.1711722166512821)]
Topic  1
[('social sector', 0.3050498240820449), ('proseguir', 0.24253651599245704), ('relanzar', 0.24072843663065366), ('cana', 0.2218025050439511), ('bienestar salud', 0.22017070618617077), ('eje economico', 0.2064935574100022), ('muni digital', 0.19857939930665464), ('juegos comunales', 0.19527243440101266), ('cantonalse', 0.192020692004463), ('sistema computo', 0.18830660998235196)]
Topic  2
[('evaluar', 1.6546902462587814), ('plp', 1.30532230318389), ('ultimos', 1.2248284407700392), ('promueve', 1.1070170275110451), ('satisfaccion', 1.029185280059857), ('jose', 0.9960766349660064), ('continuo', 0.9489400

# Alles over again

In [50]:
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.matutils import Sparse2Corpus
from gensim.models.ldamodel import LdaModel

In [66]:
# Step 1: Convert Sparse Matrix to Gensim's Corpus
corpus = Sparse2Corpus(dtm, documents_columns=False)  # dtm is the document-term matrix from Scikit-Learn TF-IDF in the previous chunk

# Step 2: Create Dictionary object that maps IDs to words
id2word = corpora.Dictionary()
id2word.token2id = tfidf_vectorizer.vocabulary_
id2word.id2token = {v: k for k, v in tfidf_vectorizer.vocabulary_.items()}

# Since the vocabulary_ of TfidfVectorizer does not preserve order, we need to sort it
sorted_vocab = sorted(tfidf_vectorizer.vocabulary_.items(), key=lambda x: x[1])
id2word.token2id = {key: val for key, val in sorted_vocab}
id2word.id2token = {val: key for key, val in sorted_vocab}

# Note: This assumes you can tokenize your texts in the same way they were tokenized for the original LDA
lda_gensim = LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=5,
    random_state=0,
    passes=10,
    alpha='auto',
    eta='auto'
)

In [65]:
# Printing topics

def display_topics(model, num_words=10):
    for topic_id, topic in model.print_topics(num_topics=-1, num_words=num_words):
        print(f"Topic {topic_id}:")
        print(topic)
        print("\n")

display_topics(lda_gensim, num_words=10)

Topic 0:
0.000*"elementos especificos" + 0.000*"llevandolo" + 0.000*"salvemos" + 0.000*"elementos" + 0.000*"especificos" + 0.000*"estrategias" + 0.000*"promover" + 0.000*"colaboracion" + 0.000*"comunidad" + 0.000*"locales"


Topic 1:
0.000*"relanzar" + 0.000*"crear" + 0.000*"promover" + 0.000*"personas" + 0.000*"coordinacion" + 0.000*"instituciones" + 0.000*"implementar" + 0.000*"programas" + 0.000*"cada" + 0.000*"municipalidad"


Topic 2:
0.000*"gobierno" + 0.000*"gobierno municipalidad" + 0.000*"promover" + 0.000*"comunidad" + 0.000*"programa" + 0.000*"programas" + 0.000*"programa gobierno" + 0.000*"participacion" + 0.000*"educacion" + 0.000*"eje"


Topic 3:
0.000*"pa" + 0.000*"zo" + 0.000*"ej" + 0.000*"on" + 0.000*"cia" + 0.000*"dependen" + 0.000*"gestion" + 0.000*"municipalidad" + 0.000*"gobierno" + 0.000*"creacion"


Topic 4:
0.000*"quintanilla" + 0.000*"claudia" + 0.000*"salazar" + 0.000*"comunidad" + 0.000*"promover" + 0.000*"espacios" + 0.000*"participacion" + 0.000*"personas" 

In [62]:
# Tokenize your texts. This should match whatever tokenizer you used to prepare your DTM for LDA
tokenized_texts = [doc.split() for doc in plans['processed_text']]  # Assuming the text is space-separated tokens

# Compute Coherence Score using C_V measure
coherence_model_lda = CoherenceModel(model=lda_gensim, texts=tokenized_texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score (C_V): ', coherence_lda)


Coherence Score (C_V):  nan


c:\Users\carme\AppData\Local\Programs\Python\Python310\lib\site-packages\gensim\topic_coherence\direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in scalar divide
  m_lr_i = np.log(numerator / denominator)
c:\Users\carme\AppData\Local\Programs\Python\Python310\lib\site-packages\gensim\topic_coherence\indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in scalar divide
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


Donx ChatGPT acerca de documentos que son muy parecidos:

Yes, when documents within your corpus are very similar and cover overlapping subjects, traditional LDA can struggle to differentiate distinct topics effectively. This is because LDA relies on variations in word frequencies across different documents to discern topics. If most documents share a similar vocabulary and thematic content, the resulting topics can be less distinct and more mixed, which might not be very helpful for your analysis.

# BERTopic

In [79]:
#pip install bertopic


     ---------------------------------------- 0.0/5.2 MB ? eta -:--:--
     - -------------------------------------- 0.2/5.2 MB 4.8 MB/s eta 0:00:02
     ---- ----------------------------------- 0.6/5.2 MB 7.0 MB/s eta 0:00:01
     -------- ------------------------------- 1.1/5.2 MB 8.5 MB/s eta 0:00:01
     -------------- ------------------------- 1.9/5.2 MB 10.8 MB/s eta 0:00:01
     ----------------- ---------------------- 2.3/5.2 MB 11.2 MB/s eta 0:00:01
     --------------------- ------------------ 2.8/5.2 MB 11.1 MB/s eta 0:00:01
     ------------------------- -------------- 3.3/5.2 MB 11.7 MB/s eta 0:00:01
     ------------------------------- -------- 4.0/5.2 MB 11.2 MB/s eta 0:00:01
     ------------------------------------ --- 4.7/5.2 MB 11.6 MB/s eta 0:00:01
     ---------------------------------------  5.2/5.2 MB 11.8 MB/s eta 0:00:01
     ---------------------------------------- 5.2/5.2 MB 11.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build de

  error: subprocess-exited-with-error
  
  × Building wheel for hdbscan (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [40 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-310
      creating build\lib.win-amd64-cpython-310\hdbscan
      copying hdbscan\flat.py -> build\lib.win-amd64-cpython-310\hdbscan
      copying hdbscan\hdbscan_.py -> build\lib.win-amd64-cpython-310\hdbscan
      copying hdbscan\plots.py -> build\lib.win-amd64-cpython-310\hdbscan
      copying hdbscan\prediction.py -> build\lib.win-amd64-cpython-310\hdbscan
      copying hdbscan\robust_single_linkage_.py -> build\lib.win-amd64-cpython-310\hdbscan
      copying hdbscan\validity.py -> build\lib.win-amd64-cpython-310\hdbscan
      copying hdbscan\__init__.py -> build\lib.win-amd64-cpython-310\hdbscan
      creating build\lib.win-amd64-cpython-310\hdbscan\tests
      copying hdbscan\tests\test_flat.py

In [80]:
#Import model libraries #~1min
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic.vectorizers import ClassTfidfTransformer

ModuleNotFoundError: No module named 'bertopic'